# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# fold path

In [4]:
drive = 'D'

In [5]:
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
raw_fold = f'{drive}:/data/big2/BD3/raw'
analysis_fold = f'{drive}:/data/big2/BD3/an'
df_fold = f'{drive}:/data/big2/BD3/df'

# Load

## 등록정보(STD_CEG_CAR_MIG)

In [4]:
# exasol db
# 4m 21s
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG;")
# test 만개
# car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG WHERE ROWNUM <= 10000;")

C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


In [5]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [6]:
carr = car.rename(columns=car_ch_col)

In [7]:
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [8]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")

In [9]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [10]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [11]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_NM  46333 non-null  object
 2   STDG_SGG_NM   46310 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [12]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [13]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [14]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## 등록이력(CEG_CAR_HISTORY_MIG)

In [15]:
# 1m 8.3s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50044247 entries, 0 to 50044246
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   VHCL_ERSR_YN  object 
 1   CHNG_DE       float64
 2   VHMNO         object 
dtypes: float64(1), object(2)
memory usage: 1.1+ GB


In [16]:
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}

In [17]:
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [18]:
carr.shape, len(carr['차대번호'].unique())

((29998033, 11), 29984639)

In [19]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(29984639, 11)

## 배출가스등급 코드 변환

In [20]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['X', '2', '3', '1', '5', '4', nan], dtype=object)

## 차종 코드 변환

In [21]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '특수', '화물', '승합', nan], dtype=object)

## 용도 코드 변환

In [22]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [23]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(26203797, 11)

## 연료 코드 변환

In [24]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '알코올', '전기', '태양열', 'CNG 하이브리드', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [25]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [26]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(26203797, 17)

## 등록&제원&법정동코드 병합(df)

In [27]:
df = cs.merge(coder, on='법정동코드', how='left')

In [28]:
df.shape

(26203797, 19)

### 매칭 안되는 지역 처리

In [29]:
df['시도'].isnull().sum()

1809

In [30]:
df.loc[df['시도'].isnull() == True, ['법정동코드', '시도', '시군구']]

,법정동코드,시도,시군구
972,4161011500,NaN,NaN
1199,4161012000,NaN,NaN
1438,4161011600,NaN,NaN
1532,4161011500,NaN,NaN
1887,4161011600,NaN,NaN
...,...,...,...
841052,4161012000,NaN,NaN
841117,4161011500,NaN,NaN
841667,4161011500,NaN,NaN
25190316,4165052000,NaN,NaN


In [31]:
df.loc[df['시도'].isnull() == True, '법정동코드'].unique()

array([4161011500, 4161012000, 4161011600, 4161011800, 4161011900,
       4161011700, 4161011400, 4165052000, 4163055000], dtype=int64)

In [32]:
# 주소 수정
df.loc[df['법정동코드'] == 4161011600, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 능평동
df.loc[df['법정동코드'] == 4161012000, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 양벌동
df.loc[df['법정동코드'] == 4161011800, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 추자동
df.loc[df['법정동코드'] == 4161011500, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 신현동
df.loc[df['법정동코드'] == 4161011900, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 매산동
df.loc[df['법정동코드'] == 4161011700, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 문형동
df.loc[df['법정동코드'] == 4161011400, ['시도', '시군구']] = ['경기도', '광주시'] # 경기도 광주시 고산동
df.loc[df['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동

In [33]:
df['시도'].isnull().sum()

0

## 등록(말소 유지) & 제원 병합

In [34]:
# 10.3s
cse = carr.merge(srcr, on='제원관리번호', how='left')

## 등록&제원&이력 병합

In [35]:
# 2m 6.0s
ersr = cse.merge(hisr, on='차량관리번호', how='left')

In [36]:
ersr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호', '차량연식',
       '차종', '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식',
       '엔진형식', '차량말소YN_y', '변경일자'],
      dtype='object')

In [37]:
# 1. 등록 차량말소와 등록이력 차량 말소 둘 모두 해당되는 데이터 추출
ersr = ersr.loc[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [39]:
# 2. 변경일자 기준 최신 데이터만 남기고 차대번호 중복 제거
ersr = ersr.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [41]:
# 3. 변경일자 2019.01.01 이후만 추출
ersr = ersr[ersr['변경일자'] >= 20190101].reset_index(drop=True)

## 등록&제원&이력&법정동코드 병합(errc)

In [43]:
# 매칭 후 '시도' 빈값 없음
errc = ersr.merge(coder, on='법정동코드', how='left')

# 분석

In [51]:
# 수도권 : 서울특별시, 인천광역시, 경기도
# 비수도권 : 수도권 외
df.loc[(df['시도'] == '서울특별시') | (df['시도'] == '인천광역시') | (df['시도'] == '경기도'), '지역'] = '수도권'
df['지역'] = df['지역'].fillna('비수도권')

In [64]:
dfm = df.copy()

In [65]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

In [66]:
# 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

In [72]:
# 수도권 : 서울특별시, 인천광역시, 경기도
# 비수도권 : 수도권 외
errc.loc[(errc['시도'] == '서울특별시') | (errc['시도'] == '인천광역시') | (errc['시도'] == '경기도'), '지역'] = '수도권'
errc['지역'] = errc['지역'].fillna('비수도권')

In [69]:
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

In [70]:
# 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

## 등급, 지역별 차량현황

In [77]:
# 2022년 차량 대수
grp1 = dfm.groupby(['지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', '지역', '시도', '배출가스등급', '차량대수']]
grp1

,연도,지역,시도,배출가스등급,차량대수
0,2022,비수도권,강원도,1,80078
1,2022,비수도권,강원도,2,358889
2,2022,비수도권,강원도,3,310193
3,2022,비수도권,강원도,4,56475
4,2022,비수도권,강원도,5,48913
...,...,...,...,...,...
97,2022,수도권,인천광역시,2,796143
98,2022,수도권,인천광역시,3,580920
99,2022,수도권,인천광역시,4,81007
100,2022,수도권,인천광역시,5,55396


In [59]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
rgn_list = []
ctpv_list = []
grd_list = []
for ctpv in grp1['시도'].unique():
    if ('서울특별시' in ctpv) or ('인천광역시' in ctpv) or ('경기도' in ctpv):
        rgn = '수도권'
    else:
        rgn = '비수도권'
        for grd in ['1', '2', '3', '4', '5', 'X']:
            for yr in ['2019', '2020', '2021', '2022']:
                yr_list.append(yr)
                rgn_list.append(rgn)
                ctpv_list.append(ctpv)
                grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, '지역':rgn_list, '시도':ctpv_list, '배출가스등급':grd_list})

In [61]:
base.head()

,연도,지역,시도,배출가스등급
0,2019,비수도권,강원도,1
1,2020,비수도권,강원도,1
2,2021,비수도권,강원도,1
3,2022,비수도권,강원도,1
4,2019,비수도권,강원도,2


In [68]:
# 연도별 등록대수
grp2 = dfm.groupby(['최초등록일자_년', '지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,지역,시도,배출가스등급,등록대수
0,1950,수도권,경기도,5,1
1,1955,비수도권,강원도,5,1
2,1955,비수도권,강원도,X,1
3,1955,비수도권,전라북도,X,1
4,1955,비수도권,충청북도,X,1
...,...,...,...,...,...
3583,2023,수도권,서울특별시,X,29405
3584,2023,수도권,인천광역시,1,8305
3585,2023,수도권,인천광역시,2,14553
3586,2023,수도권,인천광역시,3,4722


In [75]:
# 연도별 말소대수
grp3 = errc.groupby(['변경일자_년', '지역', '시도', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,지역,시도,배출가스등급,말소대수
0,2019,비수도권,강원도,1,220
1,2019,비수도권,강원도,2,4214
2,2019,비수도권,강원도,3,8206
3,2019,비수도권,강원도,4,2583
4,2019,비수도권,강원도,5,11881
...,...,...,...,...,...
403,2022,수도권,인천광역시,2,9893
404,2022,수도권,인천광역시,3,10944
405,2022,수도권,인천광역시,4,3146
406,2022,수도권,인천광역시,5,4534


In [78]:
base1 = base.merge(grp1, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base1

,연도,지역,시도,배출가스등급,차량대수
0,2019,비수도권,강원도,1,NaN
1,2020,비수도권,강원도,1,NaN
2,2021,비수도권,강원도,1,NaN
3,2022,비수도권,강원도,1,80078.00
4,2019,비수도권,강원도,2,NaN
...,...,...,...,...,...
331,2022,비수도권,충청북도,5,42772.00
332,2019,비수도권,충청북도,X,NaN
333,2020,비수도권,충청북도,X,NaN
334,2021,비수도권,충청북도,X,NaN


In [80]:
base2 = base1.merge(grp2, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base2

,연도,지역,시도,배출가스등급,차량대수,등록대수
0,2019,비수도권,강원도,1,NaN,7882.00
1,2020,비수도권,강원도,1,NaN,11940.00
2,2021,비수도권,강원도,1,NaN,14037.00
3,2022,비수도권,강원도,1,80078.00,10891.00
4,2019,비수도권,강원도,2,NaN,23262.00
...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,42772.00,1.00
332,2019,비수도권,충청북도,X,NaN,567.00
333,2020,비수도권,충청북도,X,NaN,852.00
334,2021,비수도권,충청북도,X,NaN,924.00


In [81]:
base3 = base2.merge(grp3, on=['연도', '지역', '시도', '배출가스등급'], how='left')
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원도,1,NaN,7882.00,220
1,2020,비수도권,강원도,1,NaN,11940.00,461
2,2021,비수도권,강원도,1,NaN,14037.00,854
3,2022,비수도권,강원도,1,80078.00,10891.00,501
4,2019,비수도권,강원도,2,NaN,23262.00,4214
...,...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,42772.00,1.00,4949
332,2019,비수도권,충청북도,X,NaN,567.00,106
333,2020,비수도권,충청북도,X,NaN,852.00,111
334,2021,비수도권,충청북도,X,NaN,924.00,135


In [83]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

차량대수    0
등록대수    0
말소대수    0
dtype: int64

In [84]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [85]:
base3

,연도,지역,시도,배출가스등급,차량대수,등록대수,말소대수
0,2019,비수도권,강원도,1,45026.00,7882.00,220
1,2020,비수도권,강원도,1,56505.00,11940.00,461
2,2021,비수도권,강원도,1,69688.00,14037.00,854
3,2022,비수도권,강원도,1,80078.00,10891.00,501
4,2019,비수도권,강원도,2,316079.00,23262.00,4214
...,...,...,...,...,...,...,...
331,2022,비수도권,충청북도,5,42772.00,1.00,4949
332,2019,비수도권,충청북도,X,13634.00,567.00,106
333,2020,비수도권,충청북도,X,14375.00,852.00,111
334,2021,비수도권,충청북도,X,15164.00,924.00,135


In [86]:
base3.columns

Index(['연도', '지역', '시도', '배출가스등급', '차량대수', '등록대수', '말소대수'], dtype='object')

In [87]:
df1 = base3[['연도', '지역', '시도', '배출가스등급', '차량대수']]

In [88]:
cdict = {
    '연도':'YR', 
    '지역':'RGN', 
    '시도':'CTPV', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}

In [89]:
STD_BD_CAR_CURSTT_MOD = df1.rename(columns=cdict)
STD_BD_CAR_CURSTT_MOD.columns

Index(['YR', 'RGN', 'CTPV', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT'], dtype='object')

### [출력] STD_BD_CAR_CURSTT_MOD

In [90]:
expdf = STD_BD_CAR_CURSTT_MOD
table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()

In [91]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769745696755023872 stmt_idx=1>

In [92]:
# 데이터 추가
# 5s
expdf = STD_BD_CAR_CURSTT_MOD
table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 등급, 연료별 차량현황

In [99]:
# 2022년 차량 대수
grp1 = dfm.groupby(['연료', '배출가스등급'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', '연료', '배출가스등급', '차량대수']]

In [100]:
grp1.head()

,연도,연료,배출가스등급,차량대수
0,2022,CNG 하이브리드,1,287
1,2022,CNG 하이브리드,2,17
2,2022,CNG(압축천연가스),2,24313
3,2022,CNG(압축천연가스),3,6055
4,2022,CNG(압축천연가스),4,1


In [97]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['연료'].unique():
    for grd in ['1', '2', '3', '4', '5', 'X']:
        for yr in ['2019', '2020', '2021', '2022']:
            yr_list.append(yr)
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '배출가스등급':grd_list})

In [98]:
base.head()

,연도,연료,배출가스등급
0,2019,CNG 하이브리드,1
1,2020,CNG 하이브리드,1
2,2021,CNG 하이브리드,1
3,2022,CNG 하이브리드,1
4,2019,CNG 하이브리드,2


In [101]:
# 연도별 등록대수
grp2 = dfm.groupby(['최초등록일자_년', '연료', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,연료,배출가스등급,등록대수
0,1955,경유,5,1
1,1955,기타연료,X,1
2,1956,기타연료,X,1
3,1958,휘발유,5,1
4,1962,경유,5,8
...,...,...,...,...
661,2023,휘발유,2,140120
662,2023,휘발유,X,114950
663,2023,휘발유 하이브리드,1,39268
664,2023,휘발유 하이브리드,2,17506


In [102]:
# 연도별 말소대수
grp3 = errc.groupby(['변경일자_년', '연료', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,연료,배출가스등급,말소대수
0,2019,CNG(압축천연가스),2,615
1,2019,CNG(압축천연가스),3,1257
2,2019,CNG(압축천연가스),4,4
3,2019,CNG(압축천연가스),X,26
4,2019,LPG 하이브리드,1,326
...,...,...,...,...
113,2022,휘발유,5,206
114,2022,휘발유,X,1294
115,2022,휘발유 하이브리드,1,2926
116,2022,휘발유 하이브리드,2,56


In [103]:
base1 = base.merge(grp1, on=['연도', '연료', '배출가스등급'], how='left')
base1

,연도,연료,배출가스등급,차량대수
0,2019,CNG 하이브리드,1,NaN
1,2020,CNG 하이브리드,1,NaN
2,2021,CNG 하이브리드,1,NaN
3,2022,CNG 하이브리드,1,287.00
4,2019,CNG 하이브리드,2,NaN
...,...,...,...,...
355,2022,휘발유 하이브리드,5,NaN
356,2019,휘발유 하이브리드,X,NaN
357,2020,휘발유 하이브리드,X,NaN
358,2021,휘발유 하이브리드,X,NaN


In [105]:
base2 = base1.merge(grp2, on=['연도', '연료', '배출가스등급'], how='left')
base2

,연도,연료,배출가스등급,차량대수,등록대수
0,2019,CNG 하이브리드,1,NaN,NaN
1,2020,CNG 하이브리드,1,NaN,NaN
2,2021,CNG 하이브리드,1,NaN,NaN
3,2022,CNG 하이브리드,1,287.00,NaN
4,2019,CNG 하이브리드,2,NaN,11.00
...,...,...,...,...,...
355,2022,휘발유 하이브리드,5,NaN,NaN
356,2019,휘발유 하이브리드,X,NaN,64.00
357,2020,휘발유 하이브리드,X,NaN,95.00
358,2021,휘발유 하이브리드,X,NaN,90.00


In [106]:
base3 = base2.merge(grp3, on=['연도', '연료', '배출가스등급'], how='left')
base3

,연도,연료,배출가스등급,차량대수,등록대수,말소대수
0,2019,CNG 하이브리드,1,NaN,NaN,NaN
1,2020,CNG 하이브리드,1,NaN,NaN,1.00
2,2021,CNG 하이브리드,1,NaN,NaN,6.00
3,2022,CNG 하이브리드,1,287.00,NaN,15.00
4,2019,CNG 하이브리드,2,NaN,11.00,NaN
...,...,...,...,...,...,...
355,2022,휘발유 하이브리드,5,NaN,NaN,NaN
356,2019,휘발유 하이브리드,X,NaN,64.00,7.00
357,2020,휘발유 하이브리드,X,NaN,95.00,22.00
358,2021,휘발유 하이브리드,X,NaN,90.00,12.00


In [107]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [108]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [109]:
base3

,연도,연료,배출가스등급,차량대수,등록대수,말소대수
0,2019,CNG 하이브리드,1,309.00,0.00,0.00
1,2020,CNG 하이브리드,1,308.00,0.00,1.00
2,2021,CNG 하이브리드,1,302.00,0.00,6.00
3,2022,CNG 하이브리드,1,287.00,0.00,15.00
4,2019,CNG 하이브리드,2,17.00,11.00,0.00
...,...,...,...,...,...,...
355,2022,휘발유 하이브리드,5,0.00,0.00,0.00
356,2019,휘발유 하이브리드,X,52874.00,64.00,7.00
357,2020,휘발유 하이브리드,X,52947.00,95.00,22.00
358,2021,휘발유 하이브리드,X,53025.00,90.00,12.00


In [110]:
df2 = base3[['연도', '연료', '배출가스등급', '차량대수']]

In [111]:
cdict = {
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량대수':'VHCL_MKCNT',
}

In [112]:
STD_BD_CAR_CURSTT_MOD2 = df2.rename(columns=cdict)
STD_BD_CAR_CURSTT_MOD2.columns

Index(['YR', 'FUEL_CD', 'EXHST_GAS_GRD_CD', 'VHCL_MKCNT'], dtype='object')

### [출력] STD_BD_CAR_CURSTT_MOD2

In [113]:
expdf = STD_BD_CAR_CURSTT_MOD2
table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()

In [114]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769745696755023872 stmt_idx=3>

In [115]:
# 데이터 추가
# 5s
expdf = STD_BD_CAR_CURSTT_MOD2
table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 전체 차량 현황(등급, 차종, 지역별 차량현황)

In [123]:
dfm.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '차량관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       '시도', '시군구', '지역', '최초등록일자_년', '최초등록일자_월', '최초등록일자_일', '시군구_수정'],
      dtype='object')

In [124]:
df3 = dfm[[
    '법정동코드', 
    '차종',
    '용도', 
    '차대번호', 
    '제원관리번호', 
    '배출가스인증번호', 
    '배출가스등급', 
    '차량말소YN', 
    '연료', 
    '시도', 
    '시군구_수정'
]]

In [125]:
cdict = {
    '법정동코드':'STDG_CD', 
    '차종':'VHCTY_CD',
    '용도':'PURPS_CD2', 
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO', 
    '배출가스등급':'EXHST_GAS_GRD_CD', 
    '차량말소YN':'VHCL_ERSR_YN', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
}

In [126]:
STD_BD_CAR_CURSTT = df3.rename(columns=cdict)
STD_BD_CAR_CURSTT.columns

Index(['VIN', 'STDG_CD', 'EXHST_GAS_GRD_CD', 'EXHST_GAS_CERT_NO',
       'VHCL_ERSR_YN', 'MANG_MNG_NO', '차량연식', 'VHCTY_CD', 'PURPS_CD2',
       '최초등록일자', '차량관리번호', 'FUEL_CD', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식',
       'CTPV', '시군구', '지역', '최초등록일자_년', '최초등록일자_월', '최초등록일자_일', 'SGG'],
      dtype='object')

### [출력] STD_BD_CAR_CURSTT

In [127]:
expdf = STD_BD_CAR_CURSTT
table_nm = 'STD_BD_CAR_CURSTT'.upper()

In [128]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769745696755023872 stmt_idx=5>

In [129]:
# 데이터 추가
# 3m 51.1s
expdf = STD_BD_CAR_CURSTT
table_nm = 'STD_BD_CAR_CURSTT'.upper()
wbt.import_from_pandas(expdf, table_nm)

## 지역, 연료, 연도별 차량 현황 분석

In [ ]:
# 2022년 차량 대수
grp1 = dfm.groupby(['연료', '시도'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', '연료', '시도', '차량대수']]

In [ ]:
grp1.head()

,연도,연료,배출가스등급,차량대수
0,2022,CNG 하이브리드,1,287
1,2022,CNG 하이브리드,2,17
2,2022,CNG(압축천연가스),2,24313
3,2022,CNG(압축천연가스),3,6055
4,2022,CNG(압축천연가스),4,1


In [ ]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
fuel_list = []
grd_list = []
for fuel in grp1['연료'].unique():
    for grd in ['1', '2', '3', '4', '5', 'X']:
        for yr in ['2019', '2020', '2021', '2022']:
            yr_list.append(yr)
            fuel_list.append(fuel)
            grd_list.append(grd)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '배출가스등급':grd_list})

In [ ]:
base.head()

,연도,연료,배출가스등급
0,2019,CNG 하이브리드,1
1,2020,CNG 하이브리드,1
2,2021,CNG 하이브리드,1
3,2022,CNG 하이브리드,1
4,2019,CNG 하이브리드,2


In [ ]:
# 연도별 등록대수
grp2 = dfm.groupby(['최초등록일자_년', '연료', '배출가스등급'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,연료,배출가스등급,등록대수
0,1955,경유,5,1
1,1955,기타연료,X,1
2,1956,기타연료,X,1
3,1958,휘발유,5,1
4,1962,경유,5,8
...,...,...,...,...
661,2023,휘발유,2,140120
662,2023,휘발유,X,114950
663,2023,휘발유 하이브리드,1,39268
664,2023,휘발유 하이브리드,2,17506


In [ ]:
# 연도별 말소대수
grp3 = errc.groupby(['변경일자_년', '연료', '배출가스등급'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,연료,배출가스등급,말소대수
0,2019,CNG(압축천연가스),2,615
1,2019,CNG(압축천연가스),3,1257
2,2019,CNG(압축천연가스),4,4
3,2019,CNG(압축천연가스),X,26
4,2019,LPG 하이브리드,1,326
...,...,...,...,...
113,2022,휘발유,5,206
114,2022,휘발유,X,1294
115,2022,휘발유 하이브리드,1,2926
116,2022,휘발유 하이브리드,2,56


In [ ]:
base1 = base.merge(grp1, on=['연도', '연료', '배출가스등급'], how='left')
base1

,연도,연료,배출가스등급,차량대수
0,2019,CNG 하이브리드,1,NaN
1,2020,CNG 하이브리드,1,NaN
2,2021,CNG 하이브리드,1,NaN
3,2022,CNG 하이브리드,1,287.00
4,2019,CNG 하이브리드,2,NaN
...,...,...,...,...
355,2022,휘발유 하이브리드,5,NaN
356,2019,휘발유 하이브리드,X,NaN
357,2020,휘발유 하이브리드,X,NaN
358,2021,휘발유 하이브리드,X,NaN


In [ ]:
base2 = base1.merge(grp2, on=['연도', '연료', '배출가스등급'], how='left')
base2

,연도,연료,배출가스등급,차량대수,등록대수
0,2019,CNG 하이브리드,1,NaN,NaN
1,2020,CNG 하이브리드,1,NaN,NaN
2,2021,CNG 하이브리드,1,NaN,NaN
3,2022,CNG 하이브리드,1,287.00,NaN
4,2019,CNG 하이브리드,2,NaN,11.00
...,...,...,...,...,...
355,2022,휘발유 하이브리드,5,NaN,NaN
356,2019,휘발유 하이브리드,X,NaN,64.00
357,2020,휘발유 하이브리드,X,NaN,95.00
358,2021,휘발유 하이브리드,X,NaN,90.00


In [ ]:
base3 = base2.merge(grp3, on=['연도', '연료', '배출가스등급'], how='left')
base3

,연도,연료,배출가스등급,차량대수,등록대수,말소대수
0,2019,CNG 하이브리드,1,NaN,NaN,NaN
1,2020,CNG 하이브리드,1,NaN,NaN,1.00
2,2021,CNG 하이브리드,1,NaN,NaN,6.00
3,2022,CNG 하이브리드,1,287.00,NaN,15.00
4,2019,CNG 하이브리드,2,NaN,11.00,NaN
...,...,...,...,...,...,...
355,2022,휘발유 하이브리드,5,NaN,NaN,NaN
356,2019,휘발유 하이브리드,X,NaN,64.00,7.00
357,2020,휘발유 하이브리드,X,NaN,95.00,22.00
358,2021,휘발유 하이브리드,X,NaN,90.00,12.00


In [ ]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [ ]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [ ]:
base3

,연도,연료,배출가스등급,차량대수,등록대수,말소대수
0,2019,CNG 하이브리드,1,309.00,0.00,0.00
1,2020,CNG 하이브리드,1,308.00,0.00,1.00
2,2021,CNG 하이브리드,1,302.00,0.00,6.00
3,2022,CNG 하이브리드,1,287.00,0.00,15.00
4,2019,CNG 하이브리드,2,17.00,11.00,0.00
...,...,...,...,...,...,...
355,2022,휘발유 하이브리드,5,0.00,0.00,0.00
356,2019,휘발유 하이브리드,X,52874.00,64.00,7.00
357,2020,휘발유 하이브리드,X,52947.00,95.00,22.00
358,2021,휘발유 하이브리드,X,53025.00,90.00,12.00


# code end